This project has the aim of forecasting sales numbers with the help of machine learning algorithms. It has several variations like daily foreast, daiy forecast with zip code details and monthly forecast. 

Normally, classical statistical approaches are used to forecast time series data. But there are several approaches to use machine learning and deep learning algorithms for this purpose. In this project, I tried to use both of them in my experiments.

First task, and detail of this Jupyter Notebook, is to preprocess the raw data into a suitable ML format. The raw data I acquired was in the tabular form of rows as combination and columns as historical days. For a 4 year timespan, it had about 1500 columns. 

Initial approach was to transform this input data into a moving months format. Afterwards, adding additional input variables that may be useful for predicting the sales of target month. 

First of all, raw data format must be in a standard form as code is working in a static way to transform it. There must be the data related to combination of from-to countries and then historical sales date in columnwise order. Days to be forecasted must be included at the end of the table.

Code transforms this columnwise format to a rowwise format. Parametricaly, you choose "how many past days do you want to include as input columns". 

For example, if you want to predict day N and if you choose this parameter as X, you will include all days between N-X/N-1 as input variables to predict day N sales. It is the classical approach of forecasting actually. 

If we have 1500 days of historical data; for each combination we will have about 1500 rows of input data with this format, each row corresponds to a different day with past X days as inputs to predict that specific day, in a moving pattern.

In [25]:
import pandas as pd
import numpy as np
import datetime as dt
import time
import re
from pandas.tseries.offsets import BDay
pd.set_option('display.max_columns',50)

In [26]:
#test = pd.read_excel(r'C:\Users\ali.kilinc\Desktop\Tahminleme\GerçekleşenGünlükZip - Copy.xlsx', sheet_name = 'Gerçekleşen2')

#these are very large files, first i read from excel and write to csv, then read from that csv file.

#test.to_csv(r'C:\Users\ali.kilinc\Desktop\Tahminleme\GerçekleşenGünlükZip.csv', sep='\t', encoding='utf-8', index = False)
test = pd.read_csv(r'C:\Users\ali.kilinc\Desktop\Tahminleme\GerçekleşenGünlükZip.csv', delimiter = '\t')

In [39]:
test.replace(np.nan, 0, inplace = True)

preddays = 28 #how many days to be predicted
param = 28 #moving day number
test.iloc[:,-(preddays+1):] = 0

#sums a specific value rowwise, then divides by total nb of available columns
test['zeros'] = ((test == 0).astype(int).sum(axis=1) - (preddays+1)) / (len(test.columns) - (preddays+1+5))

#(df == 0).astype(int) for boolean
#df.apply( lambda s : s.value_counts().get(0,0), axis=1) #alternative

#total sales for last 4 weeks (+2 is for one extra day column and zeros column)
test['lastdays'] = test.iloc[:,-(preddays+56+1+1):-(preddays+1+1)].sum(axis=1)

#conditional dataframe filtering according to columns
ignored = test[(test['lastdays'] == 0) | (test['zeros'] > 0.90)]

test = test[(test['lastdays'] != 0) & (test['zeros'] <= 0.90)]
#test = test.iloc[0:20,:]

print(test.info())
print(test.head(5))

test.drop(['lastdays','zeros'], axis = 1, inplace = True)
#test = test.drop(['lastdays'], axis = 1)

ignored.drop(['lastdays','zeros'], axis=1, inplace = True)
#ignored = ignored.drop(['lastdays'], axis=1)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 659 entries, 13 to 14809
Columns: 1767 entries, Servis to lastdays
dtypes: float64(1733), int64(29), object(5)
memory usage: 8.9+ MB
None
     Servis Yükleme Ülkesi Yükleme Zip Teslim Ülkesi Teslim Zip  2.01.2015  \
13  İhracat     Yunanistan          18       Polonya         47        0.0   
73  İhracat        Türkiye           1       Belçika         50        0.0   
75  İhracat        Türkiye           1       Belçika         90        0.0   
87  İhracat        Türkiye           1       Almanya         68        0.0   
90  İhracat        Türkiye           1       İspanya         08        0.0   

    3.01.2015  4.01.2015  5.01.2015  6.01.2015  7.01.2015  8.01.2015  ...  \
13   0.000000        0.0        0.0        0.0        0.0        0.0  ...   
73   0.353909        0.0        0.0        0.0        0.0        0.0  ...   
75   0.000000        0.0        0.0        0.0        0.0        0.0  ...   
87   0.000000        0.0        0.0

Above code defines the moving day and days to be predicted parameters. 

It also calculates total zero sales days from the past and last 2 month total sales. Combinations with rare sales and not very active ones will be filtered from the dataset. Otherwise it will be a huge dataset because of these high 0 percentage combinations.

In [28]:
colnum = len(test.columns)
rownum = len(test)

#list of all columns
allcols = list(test.columns.values)

# we set the first 6 fixed cols in this list. Then append several others according to param, to the column list
cols = ['service', 'from', 'from_zip', 'to', 'to_zip', 'date']
for p in range(max(preddays,param)+1):
    if p < max(preddays,param):
        cols.append('D'+'-'+str(max(preddays,param)-p)) 
    else:
        cols.append('D')
print(cols)

['service', 'from', 'from_zip', 'to', 'to_zip', 'date', 'D-28', 'D-27', 'D-26', 'D-25', 'D-24', 'D-23', 'D-22', 'D-21', 'D-20', 'D-19', 'D-18', 'D-17', 'D-16', 'D-15', 'D-14', 'D-13', 'D-12', 'D-11', 'D-10', 'D-9', 'D-8', 'D-7', 'D-6', 'D-5', 'D-4', 'D-3', 'D-2', 'D-1', 'D']


Above code sets the column names to be used after transformation of raw data

Below code creates lists from raw rowwise data in a moving pattern according to LastN Month parameter. Firstly it sets static columns for that row in each list; then it sets the date parameter then it gets Last N months + day to be predicted. 

After it completed all operations on raw data rows, dateset is turned into a Dataframe from list of lists.

In [29]:
#out = pd.DataFrame(columns = cols)
mid = []
outlist = []

# this appends lists into a list, output is list of lists. ınner lists are moving historical values with same country combinations, until the end of initial row.         
for i in range(rownum):
    rowcnt = 0
    colcnt = 0
    shift = 0
    for j in range(colnum - max(preddays,param) - 5):
        while colcnt < 5:
            mid.append(test.iloc[i,colcnt])
            colcnt = colcnt + 1
        if colcnt == 5:
            mid.append(allcols[colcnt+j+max(preddays,param)]) #it gets the column name of prediction day from column list
            colcnt = colcnt + 1
        for m in range(5+shift,5+max(preddays,param)+shift+1):
                if m < 5 + max(preddays,param) + shift:
                    mid.append(test.iloc[i,m])
                else:
                    mid.append(test.iloc[i,m]) 
        shift = shift + 1
        rowcnt = rowcnt + 1
        colcnt = 0
        outlist.append(mid)
        mid = []

#making a dataframe from list of lists
out = pd.DataFrame(outlist, columns = cols)

In [30]:
out['date'] = pd.to_datetime(out['date'], format = '%d.%m.%Y') #this one transforms to yyyy mm dd format in default. datetime type cannot be converted into mm - yyyyy
#out['date'] = out['date'].dt.to_period('m') # this one converts datetime type into period format and it is shown as yyyy mm
#out['date2'] = out['date'].dt.strftime('%m-%Y') #this one converts it into a string object which is shown as mm yyyy

out['year'] = out['date'].dt.year.astype(str)
out['quarter'] = out['date'].dt.quarter.astype(str)
out['month'] = out['date'].dt.month.astype(str)
out['week'] = out['date'].dt.week.astype(str)
out['day'] = out['date'].dt.day.astype(str)
out['weekday'] = out['date'].dt.weekday.astype(str)

In [31]:
cntdict = pd.read_excel(r'C:\Users\ali.kilinc\Desktop\Tahminleme\Gerçekleşen.xlsx', sheet_name = 'CountryCode') #dataframe with 2 columns ;country vs code
cntdict = dict(zip(cntdict['CNT'], cntdict['CODE'])) # transforming it into a dictionary

out['from'] = out['from'].map(cntdict) #change a column with help of dictionary keys, return values
out['to'] = out['to'].map(cntdict)

cntlist = np.unique(out[['from', 'to']].values.ravel('K')) #get the unique values list from multiple columns

Above code transforms country names to country codes from a dictionary

Below code calculates moving averages for each row, they will also be used as inputs for prediction model.

In [32]:
for i in [7,14,21,28]:
    movinglist = []
    for col in out.columns:
    
        if ('D-' in col):
        
            if (int(re.findall('\d+', col)[0])) <= i:
                movinglist.append(col)
    if param >= i:
        out['moving'+str(i)] = (out[movinglist].sum(axis=1))/i
    else:
        pass

out.info()
print(out.head(2))
print(out.index)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1141388 entries, 0 to 1141387
Data columns (total 45 columns):
service     1141388 non-null object
from        1141388 non-null object
from_zip    1141388 non-null object
to          1141388 non-null object
to_zip      1141388 non-null object
date        1141388 non-null datetime64[ns]
D-28        1141388 non-null float64
D-27        1141388 non-null float64
D-26        1141388 non-null float64
D-25        1141388 non-null float64
D-24        1141388 non-null float64
D-23        1141388 non-null float64
D-22        1141388 non-null float64
D-21        1141388 non-null float64
D-20        1141388 non-null float64
D-19        1141388 non-null float64
D-18        1141388 non-null float64
D-17        1141388 non-null float64
D-16        1141388 non-null float64
D-15        1141388 non-null float64
D-14        1141388 non-null float64
D-13        1141388 non-null float64
D-12        1141388 non-null float64
D-11        1141388 non-null float

Below code gets historical Eur/TL and USD/TL prices from another table, makes required transformations for date variables and then joins them to the main prediction table as input variables.

In [33]:
pd.set_option('display.max_columns',25)

kur = pd.read_excel(r'C:\Users\ali.kilinc\Desktop\Tahminleme\Gerçekleşen.xlsx', sheet_name = 'Kurlar')

kur.drop(['Gün'], axis = 1, inplace =True)

#forward fill for specific columns and all rows
kur.loc[:,['Dolar', 'Euro']] = kur.loc[:,['Dolar', 'Euro']].ffill(axis = 0)
kur.loc[:,['Dolar', 'Euro']] = kur.loc[:,['Dolar', 'Euro']].bfill(axis = 0)

#getting time values from datetime object
kur['month'] = kur['Tarih'].dt.month.astype(str)
kur['quarter'] = kur['Tarih'].dt.quarter.astype(str)
kur['year'] = kur['Tarih'].dt.year.astype(str)
kur['day'] = kur['Tarih'].dt.day.astype(str)

#kur['Dolar'] = kur.groupby(['Tarih'])['Dolar'].mean()
#normally groupby changes the structure of dataframe and output is summary table, only grouped columns and aggregation
#but transfrom changes it, uses the same data structure. !!!!

##kur['Dolar'] = kur.groupby('Tarih')['Dolar'].transform(np.mean)
##kur['Euro'] = kur.groupby('Tarih')['Euro'].transform(np.mean)
##kur['Pound'] = kur.groupby('Tarih')['Pound'].transform(np.mean)

kur.drop_duplicates(inplace=True)
kur.reset_index(inplace=True, drop=True)

kur.info()

#joining two tables, selecting only specific columns from right table, join on multiple columns.
final = pd.merge(out, kur[['Dolar', 'Euro', 'day', 'month', 'year']], how = 'left', on = ['day', 'month', 'year'])
print(final.columns)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2192 entries, 0 to 2191
Data columns (total 8 columns):
Tarih      2192 non-null datetime64[ns]
Dolar      2192 non-null float64
Euro       2192 non-null float64
Pound      1669 non-null float64
month      2192 non-null object
quarter    2192 non-null object
year       2192 non-null object
day        2192 non-null object
dtypes: datetime64[ns](1), float64(3), object(4)
memory usage: 137.1+ KB
Index(['service', 'from', 'from_zip', 'to', 'to_zip', 'date', 'D-28', 'D-27',
       'D-26', 'D-25', 'D-24', 'D-23', 'D-22', 'D-21', 'D-20', 'D-19', 'D-18',
       'D-17', 'D-16', 'D-15', 'D-14', 'D-13', 'D-12', 'D-11', 'D-10', 'D-9',
       'D-8', 'D-7', 'D-6', 'D-5', 'D-4', 'D-3', 'D-2', 'D-1', 'D', 'year',
       'quarter', 'month', 'week', 'day', 'weekday', 'moving7', 'moving14',
       'moving21', 'moving28', 'Dolar', 'Euro'],
      dtype='object')


In [34]:

pd.set_option('display.max_columns',25)

holiday = pd.read_excel(r'C:\Users\ali.kilinc\Desktop\Tahminleme\tatiller.xlsx', sheet_name = 'Tatiller')

holiday['HOLIDAY'] = pd.to_datetime(holiday['HOLIDAY'], format = '%d/%m/%Y')

holiday['day'] = holiday['HOLIDAY'].dt.day.astype(str)
holiday['weekday'] = holiday['HOLIDAY'].dt.weekday
holiday['month'] = holiday['HOLIDAY'].dt.month.astype(str)
holiday['year'] = holiday['HOLIDAY'].dt.year.astype(str)

#case when structure basically, if holiday value is x then set 1 else 0
#holiday['isfriday1'] = np.where(holiday['weekday'] == 5, 1,0)

#I wanted to count FRİDAYS according to partition. i found a way by flagging fridays above. Then sum these flags over partition. Transform was required to hold original table
#holiday['fridaycnt1'] = holiday.groupby(['COUNTRY_CODE', 'month', 'year'])['isfriday1'].transform(np.sum)

#holiday['cnt'] = holiday.groupby(['COUNTRY_CODE', 'month', 'year'], as_index = False).size().relset_index(name='cnt')['cnt']
#groupby ile yapılan aggregation işlemleri özet tabloyu getiriyor. as_index yapınca değerlerin index olmasını engelliyorsun. size fonksiyonu count alıyor, bu grupta kaç satır var gibi.
#reset index de aldığın count için sütun açmanı sağlıyor. gelen sonuç groupby içindeki 3 sütunu ve sonuç sütununu içeriyor, yani 4 sütunlu bir dataframe
#istediğimiz tek bir sütun olduğu için de o dataframe içinden ilgilendiğiöiz sütunu çekerek eşliyoruz. Ama transform gerektiği için bu yapıyı kullanmadık


#holiday table shows holidays rowwise. This line counts total holidays per month
holiday['holidaycnt'] = holiday.groupby(['COUNTRY_CODE', 'month','year'])['COUNTRY_CODE'].transform('count')

#transfrom ile yapınca groupbyda yapılan özetleme işlemine girmiyor, tüm satırlara duplike de olsa aggregate değerlerini getiriyor. 
print(holiday[holiday['COUNTRY_CODE'] == 'TR'].sort_values(by=['COUNTRY_CODE', 'year','month']).head(1))

holiday.sort_values(by=['COUNTRY_CODE', 'year', 'month'])

holiday['is_big'] = np.nan

     COUNTRY_CODE    HOLIDAY day  weekday month  year  holidaycnt
1500           TR 2015-01-01   1        3     1  2015           1


Above table gets national holidays from an external table, makes some aggregations and transformations. These holidays will be calculated for each from-to country and will be used as inputs for prediction. Below code calculates holiday types. If an holiday is just one day, it is small holiday. If it has consequent days, it is big. 

For a big holiday; we look at previous and next work week and count the number of holidays. 

For a small holiday, we count the holidays inside 3 business day range.

In [35]:
for i in range (len(holiday)):
    
    #for each holiday row, this will bring the +1-1 business day range, then i will count it countrywise
    #loc and iloc i will get same results for a full dataset. 
    #But for a sub dataset, loc will get original index position, whereas iloc will get sub table position ıf the i
    sub = holiday[((holiday['HOLIDAY'] < (holiday.loc[i,'HOLIDAY'] + BDay(2))) & (holiday['HOLIDAY'] > (holiday.loc[i,'HOLIDAY'] - BDay(2))))\
                  &(~holiday['day'].isin([5,6]))]
    
    subshrt = holiday[((holiday['HOLIDAY'] < (holiday.loc[i,'HOLIDAY'] + BDay(3)))\
                     & (holiday['HOLIDAY'] > (holiday.loc[i,'HOLIDAY'] - BDay(3))))\
                     & (~holiday['day'].isin([5,6]))]
    
    sublong = holiday[((holiday['HOLIDAY'] <= (holiday.loc[i,'HOLIDAY'] + dt.timedelta(days = 11)))\
                     & (holiday['HOLIDAY'] >= (holiday.loc[i,'HOLIDAY'] - dt.timedelta(days = 7))))\
                     &(~holiday['day'].isin([5,6]))]
    
    sub['holiday_cnt'] = sub.groupby(['COUNTRY_CODE'])['HOLIDAY'].transform('count')
    subshrt['holiday_cnt'] = subshrt.groupby(['COUNTRY_CODE'])['HOLIDAY'].transform('count')
    sublong['holiday_cnt'] = sublong.groupby(['COUNTRY_CODE'])['HOLIDAY'].transform('count')
    
    holiday.loc[i,'is_big'] = np.where(sub.loc[i,'holiday_cnt'] >= 2, 1, 0)
    holiday.at[i, 'holidaycnt'] = np.where(holiday.loc[i, 'is_big'] == 0, subshrt.loc[i, 'holiday_cnt'], sublong.loc[i, 'holiday_cnt'])

print(holiday.head(5))

C:\Users\ali.kilinc\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\ali.kilinc\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\ali.kilinc\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] 

  COUNTRY_CODE    HOLIDAY day  weekday month  year  holidaycnt  is_big
0          AFG 2018-03-21  21        2     3  2018           1     0.0
1           AT 2015-01-01   1        3     1  2015           1     0.0
2           AT 2015-01-06   6        1     1  2015           1     0.0
3           AT 2015-04-06   6        0     4  2015           1     0.0
4           AT 2015-05-01   1        4     5  2015           1     0.0


In [36]:

#This creates a full schedule to calculate fridays for each month historically
dat = pd.date_range(start = '1/1/2015', end='12/12/2020')

dat = pd.DataFrame(dat, columns = ['date'])

dat['day'] = dat['date'].dt.day.astype(str)
dat['month'] = dat['date'].dt.month.astype(str)
dat['year'] = dat['date'].dt.year.astype(str)
dat['weekday'] = dat['date'].dt.weekday

dat['isfriday2'] = np.where(dat['weekday'] ==5, 1,0)
dat['fridaycnt2'] = dat.groupby(['month', 'year'], as_index = False)['isfriday2'].transform(np.sum)

dat.drop(columns = ['date', 'weekday','isfriday2', 'fridaycnt2'], inplace = True)
dat.drop_duplicates(inplace=True)
dat.reset_index(inplace=True, drop=True)

print(dat.columns)
print(dat.head(10))



#this part cross joins country list with monthly friday table. Creates full schedule for each country in list
countries = cntlist
cnt = pd.DataFrame(countries, columns = ['COUNTRY_CODE'])

#there is not a common column for both tables. We created a fictional column to join
dat['key'] = 1
cnt['key'] = 1

all = pd.merge(cnt, dat, how = 'outer', on='key')
print(all.columns)

all.drop(columns = ['key'], inplace = True)
dat.drop(columns = ['key'], inplace = True)
cnt.drop(columns = ['key'], inplace = True)

#this part merges all countries and monthly fridays table with holiday table via left join, so that we can see them integrated. 
all = pd.merge(all, holiday[['COUNTRY_CODE', 'HOLIDAY', 'day', 'weekday', 'month', 'year', 'holidaycnt', 'is_big']], 
               how = 'left', left_on = ['COUNTRY_CODE', 'day', 'month', 'year'], right_on = ['COUNTRY_CODE', 'day', 'month', 'year'])
all.drop_duplicates(inplace=True)
all.replace(np.nan, 0, inplace = True)
all.drop(columns = ['weekday','HOLIDAY'], inplace = True)

Index(['day', 'month', 'year'], dtype='object')
  day month  year
0   1     1  2015
1   2     1  2015
2   3     1  2015
3   4     1  2015
4   5     1  2015
5   6     1  2015
6   7     1  2015
7   8     1  2015
8   9     1  2015
9  10     1  2015
Index(['COUNTRY_CODE', 'key', 'day', 'month', 'year'], dtype='object')


Above code is normally constructed for monthly predition dataset. It creates a full schedule for 5 years, counts each workday inside a month etc.(code below for example) Then it counts workdays and holidays for each different day for each country to calculate work tueasdays for october for Greece as an example. It is not mainly used for daily version, but I kept it for a possible use case.

In [37]:
''' Full code for monthly version calculations
#case when structure basically, if holiday value is x then set 1 else 0
holiday['ismon1'] = np.where(holiday['weekday'] == 1, 1,0)
holiday['istue1'] = np.where(holiday['weekday'] == 2, 1,0)
holiday['iswed1'] = np.where(holiday['weekday'] == 3, 1,0)
holiday['isthurs1'] = np.where(holiday['weekday'] == 4, 1,0)
holiday['isfriday1'] = np.where(holiday['weekday'] == 5, 1,0)


#I wanted to count FRİDAYS according to partition. i found a way by flagging fridays above. Then sum these flags over partition. Transform was required to hold original table
holiday['moncnt1'] = holiday.groupby(['COUNTRY_CODE', 'month', 'year'])['ismon1'].transform(np.sum)
holiday['tuecnt1'] = holiday.groupby(['COUNTRY_CODE', 'month', 'year'])['istue1'].transform(np.sum)
holiday['wedcnt1'] = holiday.groupby(['COUNTRY_CODE', 'month', 'year'])['iswed1'].transform(np.sum)
holiday['thurscnt1'] = holiday.groupby(['COUNTRY_CODE', 'month', 'year'])['isthurs1'].transform(np.sum)
holiday['fridaycnt1'] = holiday.groupby(['COUNTRY_CODE', 'month', 'year'])['isfriday1'].transform(np.sum)

#holiday table shows holidays rowwise. This line counts total holidays per month
holiday['holidaycnt'] = holiday.groupby(['COUNTRY_CODE', 'month','year'])['COUNTRY_CODE'].transform('count')

#transfrom ile yapınca groupbyda yapılan özetleme işlemine girmiyor, tüm satırlara duplike de olsa aggregate değerlerini getiriyor. 
print(holiday[holiday['COUNTRY_CODE'] == 'TR'].sort_values(by=['COUNTRY_CODE', 'year','month']).head(100))

#This creates a full schedule to calculate fridays for each month historically
dat = pd.date_range(start = '1/1/2015', end='12/12/2020')

dat = pd.DataFrame(dat, columns = ['date'])

dat['month'] = dat['date'].dt.month.astype(str)
dat['year'] = dat['date'].dt.year.astype(str)
dat['weekday'] = dat['date'].dt.weekday

dat['ismon2'] = np.where(dat['weekday'] ==1, 1,0)
dat['istue2'] = np.where(dat['weekday'] ==2, 1,0)
dat['iswed2'] = np.where(dat['weekday'] ==3, 1,0)
dat['isthurs2'] = np.where(dat['weekday'] ==4, 1,0)
dat['isfriday2'] = np.where(dat['weekday'] ==5, 1,0)


dat['moncnt2'] = dat.groupby(['month', 'year'], as_index = False)['ismon2'].transform(np.sum)
dat['tuecnt2'] = dat.groupby(['month', 'year'], as_index = False)['istue2'].transform(np.sum)
dat['wedcnt2'] = dat.groupby(['month', 'year'], as_index = False)['iswed2'].transform(np.sum)
dat['thurscnt2'] = dat.groupby(['month', 'year'], as_index = False)['isthurs2'].transform(np.sum)
dat['fridaycnt2'] = dat.groupby(['month', 'year'], as_index = False)['isfriday2'].transform(np.sum)

dat.drop(columns=['date', 'weekday','isfriday2', 'isthurs2', 'iswed2', 'istue2', 'ismon2'], inplace = True)
dat.drop_duplicates(inplace=True)
dat.reset_index(inplace=True, drop=True)

#this part cross joins country list with monthly friday table. Creates full schedule for each country in list
countries = cntlist
cnt = pd.DataFrame(countries, columns = ['COUNTRY_CODE'])

#there is not a common column for both tables. We created a fictional column to join
dat['key'] = 1
cnt['key'] = 1

all = pd.merge(cnt, dat, how = 'outer', on='key')

all.drop(columns = ['key'], inplace = True)
dat.drop(columns = ['key'], inplace = True)
cnt.drop(columns = ['key'], inplace = True)

#this part merges all countries and monthly fridays table with holiday table via left join, so that we can see them integrated. 
all = pd.merge(all, holiday[['COUNTRY_CODE', 'month', 'year','moncnt1', 'tuecnt1', 'wedcnt1', 'thurscnt1', 'fridaycnt1', 'holidaycnt']], 
               how = 'left', left_on = ['COUNTRY_CODE', 'month', 'year'], right_on = ['COUNTRY_CODE', 'month', 'year'])

all.drop_duplicates(inplace=True)
all.replace(np.nan, 0, inplace = True)

all['moncnt'] = all['moncnt2'] - all['moncnt1']
all['tuecnt'] = all['tuecnt2'] - all['tuecnt1']
all['wedcnt'] = all['wedcnt2'] - all['wedcnt1']
all['thurscnt'] = all['thurscnt2'] - all['thurscnt1']
all['fridaycnt'] = all['fridaycnt2'] - all['fridaycnt1'] #count of work fridays, some fridays may intersect with holiday fridays

all.drop(columns = ['fridaycnt2', 'fridaycnt1', 'thurscnt2', 'thurscnt1', 'wedcnt2', 'wedcnt1', 'tuecnt2', 'tuecnt1', 'moncnt2', 'moncnt1'], inplace = True)
'''

" Full code for monthly version calculations\n#case when structure basically, if holiday value is x then set 1 else 0\nholiday['ismon1'] = np.where(holiday['weekday'] == 1, 1,0)\nholiday['istue1'] = np.where(holiday['weekday'] == 2, 1,0)\nholiday['iswed1'] = np.where(holiday['weekday'] == 3, 1,0)\nholiday['isthurs1'] = np.where(holiday['weekday'] == 4, 1,0)\nholiday['isfriday1'] = np.where(holiday['weekday'] == 5, 1,0)\n\n\n#I wanted to count FRİDAYS according to partition. i found a way by flagging fridays above. Then sum these flags over partition. Transform was required to hold original table\nholiday['moncnt1'] = holiday.groupby(['COUNTRY_CODE', 'month', 'year'])['ismon1'].transform(np.sum)\nholiday['tuecnt1'] = holiday.groupby(['COUNTRY_CODE', 'month', 'year'])['istue1'].transform(np.sum)\nholiday['wedcnt1'] = holiday.groupby(['COUNTRY_CODE', 'month', 'year'])['iswed1'].transform(np.sum)\nholiday['thurscnt1'] = holiday.groupby(['COUNTRY_CODE', 'month', 'year'])['isthurs1'].transfo

Below code joins all sub table into a final one, which will be used as an input table for main algorithm. Then it is written as csv as an input for algorithm and xlsx for analysing the output manully for a quick basic inspection.

In [38]:

final = pd.merge(final, all, how = 'left', left_on = ['from', 'day', 'month', 'year'], right_on = ['COUNTRY_CODE', 'day', 'month','year'])
print(final.columns)

final.drop(columns = ['COUNTRY_CODE'], inplace = True)

final.rename(columns = {'holidaycnt':'dptholidaycnt', 'is_big':'dpt_is_big', 'fridaycnt': 'dptfridaycnt', 'thurscnt' : 'dptthurscnt', 'wedcnt': 'dptwedcnt', 
                        'tuecnt': 'dpttuecnt', 'moncnt':'dptmoncnt'}, inplace = True)

final = pd.merge(final, all, how = 'left', left_on = ['to', 'day', 'month', 'year'], right_on = ['COUNTRY_CODE', 'day', 'month', 'year'])



final.drop(columns = ['COUNTRY_CODE'], inplace = True)

final.rename(columns = {'holidaycnt':'arvholidaycnt', 'is_big':'arv_is_big', 'fridaycnt':'arvfridaycnt', 'thurscnt': 'arvthurscnt', 'wedcnt': 'arvwedcnt',
                        'tuecnt':'arvtuecnt', 'moncnt':'arvmoncnt'}, inplace = True)
print(final.columns)
print(final.info())


#print_excel = final.to_excel(r'C:\Users\ali.kilinc\Desktop\Tahminleme\MovGünlükPrepForecastCompany28.xlsx', index = None, header = True, sheet_name = 'FullTable')
#print_csv = final.to_csv(r'C:\Users\ali.kilinc\Desktop\Tahminleme\MovGünlükPrepForecastCompany28.csv', index = None, header = True)


Index(['service', 'from', 'from_zip', 'to', 'to_zip', 'date', 'D-28', 'D-27',
       'D-26', 'D-25', 'D-24', 'D-23', 'D-22', 'D-21', 'D-20', 'D-19', 'D-18',
       'D-17', 'D-16', 'D-15', 'D-14', 'D-13', 'D-12', 'D-11', 'D-10', 'D-9',
       'D-8', 'D-7', 'D-6', 'D-5', 'D-4', 'D-3', 'D-2', 'D-1', 'D', 'year',
       'quarter', 'month', 'week', 'day', 'weekday', 'moving7', 'moving14',
       'moving21', 'moving28', 'Dolar', 'Euro', 'COUNTRY_CODE', 'holidaycnt',
       'is_big'],
      dtype='object')
Index(['service', 'from', 'from_zip', 'to', 'to_zip', 'date', 'D-28', 'D-27',
       'D-26', 'D-25', 'D-24', 'D-23', 'D-22', 'D-21', 'D-20', 'D-19', 'D-18',
       'D-17', 'D-16', 'D-15', 'D-14', 'D-13', 'D-12', 'D-11', 'D-10', 'D-9',
       'D-8', 'D-7', 'D-6', 'D-5', 'D-4', 'D-3', 'D-2', 'D-1', 'D', 'year',
       'quarter', 'month', 'week', 'day', 'weekday', 'moving7', 'moving14',
       'moving21', 'moving28', 'Dolar', 'Euro', 'dptholidaycnt', 'dpt_is_big',
       'arvholidaycnt', 'arv_

After the initial filters, combination size decreased to 650. Apllying the transformation to this input resulted in a table of 1100K rows.